In [112]:
from numpy import exp, array, random, dot
import numpy as np
from scipy import misc
import skimage
import math
from __future__ import with_statement

In [113]:
arr1 = np.divide(misc.imread('tAmerica1.png'),255)
arr1r = arr1[:,:,0]
arr1g = arr1[:,:,1]
arr1b = arr1[:,:,2]
arr2 = np.divide(misc.imread('tAmerica2.png'),255)
arr2r = arr2[:,:,0]
arr2g = arr2[:,:,1]
arr2b = arr2[:,:,2]

In [121]:
filterNum = 1
filterSize = 10
layerNum = 6

In [132]:
filters = []
for x in range(filterSize*layerNum):
    filters += [0.01*random.random((filterSize,filterSize))-0.005] #run previous before this 
filters = array(filters)
print(filters)

[[[  4.68034424e-03   4.75105425e-03   1.39762296e-03 ...,  -3.13441061e-03
     1.70822534e-03  -4.35934966e-03]
  [ -3.18894004e-03   5.36382935e-04  -1.59376028e-03 ...,   8.50538455e-04
    -4.03122868e-03  -4.74562117e-03]
  [  4.45859807e-03  -2.18594807e-04  -3.84491787e-03 ...,   1.19317204e-03
     8.27162057e-04   2.97048975e-03]
  ..., 
  [  2.43895367e-04   2.52324581e-03   3.50122085e-03 ...,  -2.69280922e-03
     1.17005460e-03   2.30254916e-04]
  [ -4.49553507e-03  -1.27770990e-03   4.23114585e-03 ...,  -4.76653775e-03
     5.59828329e-04  -4.21270954e-03]
  [  3.86185439e-03   2.84460932e-03  -2.51039738e-03 ...,  -2.29526997e-03
    -3.67518101e-03   3.02783487e-03]]

 [[  1.71454793e-03  -2.83745376e-03  -1.85282705e-03 ...,   1.13889762e-03
     3.27198616e-03  -3.10861620e-03]
  [  2.99422277e-03  -4.34810083e-03  -2.36626342e-03 ...,   4.50935955e-03
     3.35967746e-03  -4.75073969e-03]
  [ -4.94091584e-03   2.48168267e-03  -3.42579997e-03 ...,  -2.62583554e-03
  

In [123]:
def activation(x, deriv=False):
    if deriv:
        return (x > 0)
    return x * (x > 0)
def sumNoNeg(arr):
    result = 0
    for y in range(len(arr)):
        for x in range(len(arr[0])):
            result += max(arr[y][x],0)
    return result
def apply(filterArr, img, row, col):
    value = 0
    halfFil = int(filterSize/2)
    padImg = np.pad(img, [(halfFil,halfFil),(halfFil,halfFil)],'constant', constant_values=(0))
    for y in range(len(filterArr)):
        for x in range(len(filterArr[0])):
            value += padImg[y+row-halfFil][x+col-halfFil]*filterArr[y][x]
    return value/sumNoNeg(filterArr)
print(apply(filters[4],arr1r,0,0))

0.0


In [124]:
def applyFilters(img, new_img, filterArr):
    for y in range(len(img)):
        for x in range(len(img[y])):
            new_img[y][x] += activation(apply(filterArr,img,y,x))
    return new_img
# print(len(([[0]*(len(arr1r))]*(len(arr1r[0])))[0]))
# print(len(arr1r[0]))
# #print([[0]*(len(arr1r)-filterSize)]*(len(arr1r[0])-filterSize))
# print(len(arr1r))
# print(len(arr1r[0]))
# newArr = applyFilters(arr1r, np.zeros((len(arr1r), len(arr1r[0]))), filters[0])
# print(len(newArr))
# print(len(newArr[0]))

158
300
158
300


In [126]:
def forward(img):
    outputLayers = [img]
    for num in range(layerNum):
        newImage = np.zeros((len(outputLayers[num]), len(outputLayers[num][0])))
        for fil in range(filterNum):
            applyFilters(outputLayers[num], newImage, filters[filterNum*num + fil])
            print(filterNum*num + fil)
            print(newImage)
        outputLayers.append(np.divide(newImage, filterNum))
    return outputLayers
out = forward(arr1r)
print(out)

0
[[ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 ..., 
 [ 0.          0.07661217  0.         ...,  0.05027889  0.05027889
   0.05027889]
 [ 0.          0.07636284  0.         ...,  0.06559514  0.06559514
   0.06559514]
 [ 0.          0.07636284  0.         ...,  0.06559514  0.06559514
   0.06559514]]
1
[[ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 ..., 
 [ 0.          0.          0.         ...,  0.00376113  0.00376113
   0.00376113]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]]
2
[[ 0.     

In [119]:
def maxPool(size, img):
    new_img = np.zeros((int(len(img)/size), int(len(img[0])/size)))
    pad_img = np.pad(img, [(0, size-len(img)%size),(0, size-len(img[0])%size)], "constant", constant_values=(0))
    for x in range(math.ceil(len(img)/size)-1):
        for y in range(math.ceil(len(img[0])/size)-1):
            new_img[x][y] = np.amax(array(pad_img[x*size:((x+1)*size)][y*size:((y+1)*size)]))
            print(array(pad_img[x*size:((x+1)*size)][y*size:((y+1)*size)]))
    return new_img
print(maxPool(3, out[2]))

[[  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00  

ValueError: zero-size array to reduction operation maximum which has no identity

In [130]:
def error(output):
    return 1-np.mean(output)
def back(img, startlayer, error):
    print(startlayer)
    newout = np.zeros((len(out[startlayer-1]), len(out[startlayer-1][0])))
    if startlayer == -1:
        return out[0]
    delta = applyFilters(back(img, startlayer-1, error),
                 newout,
                 np.multiply(np.rot90(np.rot90(filters[startlayer])),activation(img,deriv=True)[:filterSize,:filterSize]))
    filters[startlayer]=np.add(filters[startlayer],array(delta[:filterSize,:filterSize]))
    print(startlayer)
    return newout
backprop = back(out[layerNum], layerNum, error(out[layerNum]))

6
5
4
3
2
1
0
-1


/Users/frank/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:18: RuntimeWarning: invalid value encountered in double_scalars


0
1
2
3
4
5
6


In [131]:
print(filters)

[[[             nan              nan              nan ...,              nan
                nan              nan]
  [             nan              nan              nan ...,              nan
                nan              nan]
  [             nan              nan              nan ...,              nan
                nan              nan]
  ..., 
  [             nan              nan              nan ...,              nan
                nan              nan]
  [             nan              nan              nan ...,              nan
                nan              nan]
  [             nan              nan              nan ...,              nan
                nan              nan]]

 [[             nan              nan              nan ...,              nan
                nan              nan]
  [             nan              nan              nan ...,              nan
                nan              nan]
  [             nan              nan              nan ...,              nan
  